This notebook is a fork of https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer

In [1]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
from PIL import Image
import seaborn as sns, gc
from yolov5.utils.general import scale_boxes, xyxy2xywh

In [2]:
SUBM_FOLDER = 'images/'

In [3]:
test_img = SUBM_FOLDER + 'test.png'

In [4]:
global classes
classes = ['Aortic enlargement',
            'Atelectasis',
            'Calcification',
            'Cardiomegaly',
            'Consolidation',
            'ILD',
            'Infiltration',
            'Lung Opacity',
            'Nodule/Mass',
            'Other lesion',
            'Pleural effusion',
            'Pleural thickening',
            'Pneumothorax',
            'Pulmonary fibrosis']

# YOLOv5 Stuff

In [5]:
#shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
#os.chdir('/content/drive/MyDrive/Thesis/yolov5')
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0 _CudaDeviceProperties(name='NVIDIA GeForce GTX TITAN X', major=5, minor=2, total_memory=12204MB, multi_processor_count=24)


In [40]:
import cv2
import colorsys
import random

colors = [
    (33, 33, 33), #Dark Gray
    (47, 79, 79), #Dark Slate Gray
    (25, 25, 112), #Midnight Blue
    (0, 0, 128), #Navy Blue
    (139, 0, 0), #Dark Red
    (128, 0, 0), #Maroon
    (128, 0, 128), #Indigo
    (139, 0, 139), #Dark Magenta
    (128, 128, 0), #Olive
    (0, 128, 0), #Dark Green
    (0, 128, 128), #Teal
    (47, 79, 79), #Dark Slate Gray
    (139, 69, 19), #Saddle Brown
    (0, 100, 0), #Dark Green
]

def visualize_yolo(image_path, preds, width, height):
    print("Draw boxes!")
    global classes, colors
    image = cv2.imread(image_path)

    if len(preds) == 0:
        return image
    
    if preds[0][0] == 14:
        return image
    
    for pred in preds:
        label, score, box = pred
        color = colors[int(label)]
        label = classes[int(label)]

        # Generate a random color based on the label
        #hsv = [(hash(label) % 255) / 255.0, 1.0, 1.0]
        #color = tuple(int(255 * c) for c in colorsys.hsv_to_rgb(*hsv))
        # Extract box coordinates
        x_min, y_min, x_max, y_max = map(int, box)

        # Draw the bounding box on the image
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

        # Create the label string
        #label_text = f"{label}: {score:.2f}"
        label_text = f"{label}"

        # Get the size of the label text
        font_scale = height / 500 * 0.5
        label_size, _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)

        # Draw the label rectangle
        cv2.rectangle(image, (x_min, y_min - label_size[1] - 5),
                      (x_min + label_size[0], y_min - 0), color, cv2.FILLED)

        # Put the label text
        cv2.putText(image, label_text, (x_min, y_min - 2),
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 1)
    
    # cv2.imshow("Bounding Boxes", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return image
    # Display the image with bounding boxes
    # cv2.imshow("Bounding Boxes", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
        

# Inference

In [7]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [8]:
models = []
for i in range(0,5):
    models.append(torch.hub.load('./yolov5', 'custom', path=f'weights/best_{i}.pt', source='local'))
    models[i].iou = 0.25
    models[i].conf = 0.01


YOLOv5 🚀 v7.0-175-g5f11555 Python-3.7.16 torch-1.7.0 CUDA:0 (NVIDIA GeForce GTX TITAN X, 12204MiB)

Fusing layers... 
Model summary: 322 layers, 86260891 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.7.16 torch-1.7.0 CUDA:0 (NVIDIA GeForce GTX TITAN X, 12204MiB)

Fusing layers... 
Model summary: 322 layers, 86260891 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.7.16 torch-1.7.0 CUDA:0 (NVIDIA GeForce GTX TITAN X, 12204MiB)

Fusing layers... 
Model summary: 322 layers, 86260891 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.7.16 torch-1.7.0 CUDA:0 (NVIDIA GeForce GTX TITAN X, 12204MiB)

Fusing layers... 
Model summary: 322 layers, 86260891 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.7.16 torch-1.7.0 CUDA:0 (NVIDIA GeForce GTX TITAN X, 12204MiB)

Fusing layers... 
Model summary: 322 layers, 862

In [9]:
from PIL import Image
def detect_img(model, img, size = 640):
        results = model(img, size)

        image = Image.open(img)
        width, height = image.size

        dets = []
        for i, detect_info in enumerate(results.xyxy):
                img1shape = (height, width)
                detected_coordinates = detect_info[:, :4]
                img0shape = (height, width)
                detect_info[:, :4] = scale_boxes(img1shape, detected_coordinates, img0shape).round()
                xywhs = xyxy2xywh(detect_info[:, 0:4]).cpu().numpy()
                confs = detect_info[:, 4].cpu().numpy()
                clss = detect_info[:, 5].cpu().numpy()
                for i in reversed(list(range(0, len(clss)))):
                        tmp = [clss[i], xywhs[i][0] / width, xywhs[i][1] / height, xywhs[i][2] / width, xywhs[i][3] / height, confs[i]]
                        for num in tmp:
                                dets.append(round(num, 6))
        return dets

In [10]:
def run_yolo_5fold(test_img, width, height):
    print('Yolo 5 fold running!')
    preds = []
    for i in range(0,5):
        torch.cuda.empty_cache()
        gc.collect()

        PredictionStrings = ''

        w, h = width, height

        dets = detect_img(models[i], test_img, 640)

        data = np.array(dets).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.concatenate((data[:, :2], yolo2voc(h, w, data[:, 2:])), axis =1).reshape(-1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        PredictionStrings = ' '.join(bboxes)
        preds.append(PredictionStrings)
    return preds


## Ensemble all Yolo Folds

In [11]:
from ensemble_boxes import weighted_boxes_fusion
import pandas as pd


SUBM_PATH = './subm_folder/'

def ensemble(
    preds,
    weights,
    iou_same=0.5,
    skip_box_thr=0.00000001,
    w = 640,
    h = 640
):
    print("Ensemble 5 folds!")

    boxes_list = []
    scores_list = []
    labels_list = []
    empty = True
    for i in range(len(preds)):
        boxes = []
        scores = []
        labels = []
        p1 = preds[i]
        if str(p1) != 'nan':
            arr = p1.strip().split(' ')
            for k in range(0, len(arr), 6):
                cls = int(arr[k])
                prob = float(arr[k + 1])
                x1 = float(arr[k + 2]) / w
                y1 = float(arr[k + 3]) / h
                x2 = float(arr[k + 4]) / w
                y2 = float(arr[k + 5]) / h
                boxes.append([x1, y1, x2, y2])
                scores.append(prob)
                labels.append(cls)

        boxes_list.append(boxes)
        scores_list.append(scores)
        labels_list.append(labels)

    boxes, scores, labels = weighted_boxes_fusion(
        boxes_list,
        scores_list,
        labels_list,
        iou_thr=iou_same,
        skip_box_thr=skip_box_thr,
        weights=weights,
        allows_overflow=True
    )
    boxes = boxes.tolist()
    scores = scores.tolist()
    labels = labels.tolist()
    
    # print(len(boxes), len(labels), len(scores))
    if len(boxes) == 0:
        boxes.append([0, 0, 1, 1])
        scores.append(1)
        labels.append(14)

    return boxes, scores, labels

In [12]:
### 2 CLASS FILTER

In [13]:
from dataclasses import dataclass, field
from typing import Dict, Any, Tuple, Union, List


@dataclass
class Flags:
    # General
    debug: bool = True
    outdir: str = "results/det"
    device: str = "cuda:0"

    # Data config
    imgdir_name: str = "vinbigdata-chest-xray-resized-png-256x256"
    # split_mode: str = "all_train"  # all_train or valid20
    seed: int = 111
    target_fold: int = 0  # 0~4
    label_smoothing: float = 0.0
    # Model config
    model_name: str = "resnet18"
    model_mode: str = "normal"  # normal, cnn_fixed supported
    # Training config
    epoch: int = 20
    batchsize: int = 8
    valid_batchsize: int = 16
    num_workers: int = 4
    snapshot_freq: int = 5
    ema_decay: float = 0.999  # negative value is to inactivate ema.
    scheduler_type: str = ""
    scheduler_kwargs: Dict[str, Any] = field(default_factory=lambda: {})
    scheduler_trigger: List[Union[int, str]] = field(default_factory=lambda: [1, "iteration"])
    aug_kwargs: Dict[str, Dict[str, Any]] = field(default_factory=lambda: {})
    mixup_prob: float = -1.0  # Apply mixup augmentation when positive value is set.

    def update(self, param_dict: Dict) -> "Flags":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self

In [14]:
#!pip install pytorch-pfn-extras==0.3.2
#!pip install timm==0.3.4

In [15]:
flags = Flags()

In [16]:
from torch import nn
from torch.nn import Linear
from torch.utils.data.dataloader import DataLoader

In [17]:

class CNNFixedPredictor(nn.Module):
    def __init__(self, cnn: nn.Module, num_classes: int = 2):
        super(CNNFixedPredictor, self).__init__()
        self.cnn = cnn
        self.lin = Linear(cnn.num_features, num_classes)
        print("cnn.num_features", cnn.num_features)

        # We do not learn CNN parameters.
        # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
        for param in self.cnn.parameters():
            param.requires_grad = False

    def forward(self, x):
        feat = self.cnn(x)
        return self.lin(feat)

In [18]:
import timm


def build_predictor(model_name: str, model_mode: str = "normal"):
    if model_mode == "normal":
        # normal configuration. train all parameters.
        return timm.create_model(model_name, pretrained=True, num_classes=2, in_chans=3)
    elif model_mode == "cnn_fixed":
        # normal configuration. train all parameters.
        # https://rwightman.github.io/pytorch-image-models/feature_extraction/
        timm_model = timm.create_model(model_name, pretrained=True, num_classes=0, in_chans=3)
        return CNNFixedPredictor(timm_model, num_classes=2)
    else:
        raise ValueError(f"[ERROR] Unexpected value model_mode={model_mode}")

In [19]:
import torch
import torch.nn.functional as F


def cross_entropy_with_logits(input, target, dim=-1):
    loss = torch.sum(- target * F.log_softmax(input, dim), dim)
    return loss.mean()

In [20]:
import torch
print(torch.__version__)
import torch.nn.functional as F
from torch import nn
import pytorch_pfn_extras as ppe

class Classifier(nn.Module):
    """two class classfication"""

    def __init__(self, predictor, lossfun=cross_entropy_with_logits):
        super().__init__()
        self.predictor = predictor
        self.lossfun = lossfun
        self.prefix = ""

    def forward(self, image, targets):
        outputs = self.predictor(image)
        loss = self.lossfun(outputs, targets)
        metrics = {
            f"{self.prefix}loss": loss.item(),
            f"{self.prefix}acc": accuracy_with_logits(outputs, targets).item()
        }
        ppe.reporting.report(metrics, self)
        return loss, metrics

    def predict(self, data_loader):
        pred = self.predict_proba(data_loader)
        label = torch.argmax(pred, dim=1)
        return label

    def predict_proba(self, data_loader):
        device: torch.device = next(self.parameters()).device
        y_list = []
        self.eval()
        with torch.no_grad():
            for batch in data_loader:
                if isinstance(batch, (tuple, list)):
                    # Assumes first argument is "image"
                    batch = batch[0].to(device)
                else:
                    batch = batch.to(device)
                y = self.predictor(batch)
                y = torch.softmax(y, dim=-1)
                y_list.append(y)
        pred = torch.cat(y_list)
        return pred

fail to import apex_C: apex was not installed or installed without --cpp_ext.
fail to import amp_C: apex was not installed or installed without --cpp_ext.


1.7.0


In [21]:
"""
Referenced `chainer.dataset.DatasetMixin` to work with pytorch Dataset.
"""
import numpy
import six
import torch
from torch.utils.data.dataset import Dataset


class DatasetMixin(Dataset):

    def __init__(self, transform=None):
        self.transform = transform

    def __getitem__(self, index):
        """Returns an example or a sequence of examples."""
        if torch.is_tensor(index):
            index = index.tolist()
        if isinstance(index, slice):
            current, stop, step = index.indices(len(self))
            return [self.get_example_wrapper(i) for i in
                    six.moves.range(current, stop, step)]
        elif isinstance(index, list) or isinstance(index, numpy.ndarray):
            return [self.get_example_wrapper(i) for i in index]
        else:
            return self.get_example_wrapper(index)

    def __len__(self):
        """Returns the number of data points."""
        raise NotImplementedError

    def get_example_wrapper(self, i):
        """Wrapper of `get_example`, to apply `transform` if necessary"""
        example = self.get_example(i)
        if self.transform:
            example = self.transform(example)
        return example

    def get_example(self, i):
        """Returns the i-th example.

        Implementations should override it. It should raise :class:`IndexError`
        if the index is invalid.

        Args:
            i (int): The index of the example.

        Returns:
            The i-th example.

        """
        raise NotImplementedError

In [22]:
import cv2
import numpy as np


class VinbigdataTwoClassDataset(DatasetMixin):
    def __init__(self, dataset_dicts, image_transform=None, transform=None, train: bool = True,
                 mixup_prob: float = -1.0, label_smoothing: float = 0.0):
        super(VinbigdataTwoClassDataset, self).__init__(transform=transform)
        self.dataset_dicts = dataset_dicts
        self.image_transform = image_transform
        self.train = train
        self.mixup_prob = mixup_prob
        self.label_smoothing = label_smoothing

    def _get_single_example(self, i):
        d = self.dataset_dicts[i]
        filename = d["file_name"]

        img = cv2.imread(filename)
        if self.image_transform:
            img = self.image_transform(img)
        img = torch.tensor(np.transpose(img, (2, 0, 1)).astype(np.float32))

        if self.train:
            label = int(len(d["annotations"]) > 0)  # 0 normal, 1 abnormal
            if self.label_smoothing > 0:
                if label == 0:
                    return img, float(label) + self.label_smoothing
                else:
                    return img, float(label) - self.label_smoothing
            else:
                return img, float(label)
        else:
            # Only return img
            return img, None

    def get_example(self, i):
        img, label = self._get_single_example(i)
        if self.mixup_prob > 0. and np.random.uniform() < self.mixup_prob:
            j = np.random.randint(0, len(self.dataset_dicts))
            p = np.random.uniform()
            img2, label2 = self._get_single_example(j)
            img = img * p + img2 * (1 - p)
            if self.train:
                label = label * p + label2 * (1 - p)

        if self.train:
            label_logit = torch.tensor([1 - label, label], dtype=torch.float32)
            return img, label_logit
        else:
            # Only return img
            return img

    def __len__(self):
        return len(self.dataset_dicts)

In [23]:
def get_vinbigdata_dicts_test(img, width, height):
    dataset_dicts = []
    record = {}
    record["file_name"] = img
    # record["image_id"] = index
    record["image_id"] = img
    record["height"] = height
    record["width"] = width
    # objs = []
    # record["annotations"] = objs
    dataset_dicts.append(record)
    return dataset_dicts


In [24]:
def predict_2class(test_img, width, height):
    model = build_predictor(model_name=flags.model_name, model_mode=flags.model_mode)
    weight_path = 'weights/predictor.pt'
    state_dict = torch.load(weight_path)

    # Load your weights into the model
    model.load_state_dict(state_dict)

    classifier = Classifier(model)

    dataset_dicts_test = get_vinbigdata_dicts_test(test_img, width, height)
    test_dataset = VinbigdataTwoClassDataset(dataset_dicts_test, train=False)
    test_loader = DataLoader(
        test_dataset,
        batch_size=flags.valid_batchsize,
        num_workers=flags.num_workers,
        shuffle=False,
        pin_memory=True,
    )
    test_pred = classifier.predict_proba(test_loader).cpu().numpy()
    test_pred_df = pd.DataFrame({
        "image_id": [d["image_id"] for d in dataset_dicts_test],
        "class0": test_pred[:, 0],
        "class1": test_pred[:, 1]
    })
    return test_pred_df

In [25]:
import pandas as pd

def filter_2class(test_img, width, height):
    low_threshold = 0.0
    high_threshold = 0.9

    NORMAL = "14 1 0 0 1 1"

    test_pred_df = predict_2class(test_img, width, height)

    p0 = test_pred_df.loc[0, "class0"]
    if p0 < low_threshold:
        print('Keep')
        return True
    elif low_threshold <= p0 and p0 < high_threshold:
        #boxes.append([0, 0, 1, 1])
        #scores.append(p0)
        #labels.append(14)
        print('Add')
        return True
    else:
        # boxes = [[0, 0, 1, 1]]
        # scores = [p0]
        # labels = [14]
        print('Replace')
        return False



### Filter Cardiomegaly

In [26]:
# !python3 cardiomegaly_filter/cardiomegaly_adjusted_776.py --path ./images --threshold 0.52 --low 0.5 --high 0.55

In [27]:
from cardiomegaly_filter.cardiomegaly_detection import load_models, predict
import torch
global device, unet, unet2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

unet, unet2 = load_models(device)

In [28]:
import csv
import pandas as pd
import math

#INPUT_PATH = 'submission_ensemble.csv'
#INPUT_PATH = 'subm_folder/yolo_filter_25_001.csv'
#OUTPUT_PATH = 'yolo_filter_25_001_final.csv'
CARDIO_CLASS = 3
NO_FINDINGS_CLASS = 14
CARDIO_PATH = 'result/'
THRESHOLD_NO_FINDINGS = 0.98

global cnt
cnt = 0

def find_best_score(preds):
    if len(preds) == 0:
        return []
    
    res = preds[0]
    for class_id, score, location in preds:
        if score > res[1]:
            res = (class_id, score, location)
    
    return [res]
        
def is_cardiomegaly(filename, pred, preds):
    numbers = []
    with open(CARDIO_PATH + filename + '.txt', 'r') as file:
        for line in file:
            # Split the line into individual elements
            elements = line.strip().split()
            for element in elements:
                try:
                    number = float(element)
                    numbers.append(number)
                except ValueError:
                    pass

    class_id = numbers[0]
    score = numbers[1]
    location = numbers[2:6]
    
    flag1 = class_id == CARDIO_CLASS
    flag2 = len(pred) > 0
   
    if flag2:
        if not flag1 and score > 0.6:
            best = find_best_score(pred)[0]
            if best[1] <= 0.3:
                return preds
            else:
                return preds + pred
        elif flag1 and score > 0.6:
            best = find_best_score(pred)[0]
            preds.append((best[0], best[1] * 2, best[2]))
            return preds
        else:
            return preds + pred
    else:
        return preds + pred


def filter_cardiomegaly(labels, scores, boxes, width, height):
    filename = 'test'

    cardios = []
    no_findings = []
    preds = []

    for i, label in enumerate(labels):
        score = scores[i]
        box = boxes[i]
        
        if label == CARDIO_CLASS:
            cardios.append((label, score, box))
        else:
            preds.append((label, score, box))

        if label == NO_FINDINGS_CLASS:
            no_findings.append((label, score, box))

    # if out_row[0] != '1' or out_row[1] != '4':
    #     pred, difference = is_cardiomegaly(filename, cardios)
    #     out_row = f'{out_row}{convert2string(pred)}'
    #if len(no_findings) == 0 or find_best_score(no_findings)[0][1] < THRESHOLD_NO_FINDINGS:
    new_preds = is_cardiomegaly(filename, cardios, preds)
    final_preds = []
    for pred in new_preds:
        label, score, box = pred
        if score >= 0.1:
            final_preds.append((label, score, [box[0] * width, box[1] * height, box[2] * width, box[3] * height]))

    return final_preds

In [42]:
def predict_image(test_img, file_name):
    global image
    image = Image.open(test_img)
    width, height = image.size

    preds = run_yolo_5fold(test_img, width, height)
    best_iou = 0.3
    boxes, scores, labels = ensemble(preds, [1, 1, 1, 1, 1], best_iou, w=width, h=height)

    if not filter_2class(test_img, width, height):
        boxes = [[0, 0, 1, 1]]
        scores = [1]
        labels = [14]

    #!python3 cardiomegaly_filter/cardiomegaly_adjusted_776.py --path ./images --threshold 0.52 --low 0.5 --high 0.55
    global unet, unet2, device
    print("Predict Cardio")
    predict(unet, unet2, device, file_name)

    print("Filter Cardio")
    final_preds = filter_cardiomegaly(labels, scores, boxes, width, height)

    return visualize_yolo(test_img, final_preds, width, height)
    
    

In [43]:
import pickle
from flask import Flask, render_template, request
from werkzeug.wrappers import Request, Response
import dlib
import os
from random import random
import cv2
import numpy as np
import  sys


# Khởi tạo Flask
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = "static"

# Hàm xử lý request
@app.route("/", methods=['GET', 'POST'])
def home_page():
    # Nếu là POST (gửi file)
    if request.method == "POST":
         try:
            # Lấy file gửi lên
            image = request.files['file']
            if image:
                # Lưu file
                print(image.filename)
                filename = 'test.' + image.filename.split('.')[-1]
                path_to_save = 'images/' + filename
                print("Save = ", path_to_save)
                image.save(path_to_save)

                global test_img
                test_img = path_to_save

                pred_image = predict_image(path_to_save, filename)

                out_path = app.config['UPLOAD_FOLDER'] + "/" + filename
                cv2.imwrite(out_path, pred_image)

                    # Trả về kết quả
                return render_template("index.html", user_image = filename , rand = str(random()), 
                                       msg="Uploaded successfully!")
            else:
                # Nếu không có file thì yêu cầu tải file
                return render_template('index.html', msg='Choose file.')

         except Exception as ex:
            # Nếu lỗi thì thông báo
            print(ex)
            return render_template('index.html', msg='Something wrong!')

    else:
        # Nếu là GET thì hiển thị giao diện upload
        return render_template('index.html')


if __name__ == '__main__':
    #app.run(host='127.0.0.1', debug=True)
    from werkzeug.serving import run_simple
    run_simple('localhost', 8080, app)

 * Running on http://localhost:8080
Press CTRL+C to quit


demo.jpg
Save =  images/test.jpg
Yolo 5 fold running!
Ensemble 5 folds!
Add
Predict Cardio


127.0.0.1 - - [13/Jun/2023 01:44:35] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2023 01:44:35] "GET /static/test.jpg?0.05658256496780578 HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2023 01:44:35] "GET /static/bg.jpg HTTP/1.1" 304 -


No Heart Segment!
Filter Cardio
Draw boxes!
